# Codi

In [1]:
from collections import Counter
from typing import List, Tuple
import math
import random

Funcions de codificació i descodificació a partir d'una llista de correlacions.

In [2]:
def encode(text: str, corr: List[Tuple[str, str]]) -> str:
    map_ab = dict(corr)
    i = 0
    j = 1
    result = []
    while j <= len(text):
        word = text[i:j]
        if word in map_ab:
            result.append(map_ab[word])
            i = j
        j += 1

    return "".join(result)


def decode(text: str, corr: List[Tuple[str, str]]) -> str:
    map_ba = {b: a for (a, b) in corr}
    i = 0
    j = 1
    result = []
    while j <= len(text):
        word = text[i:j]
        if word in map_ba:
            result.append(map_ba[word])
            i = j
        j += 1

    return "".join(result)

Generació del codi canònic q-ari a partir d'una llista de longituds de les paraules-codi
`lengths` i d'un alfabet `alf`.

In [3]:
def increment_by_one(value, alf):
    n = len(alf)
    val_idx = len(value) - 1
    alf_idx = alf.index(value[val_idx])

    # Propagate increment if at last symbol of alphabet
    while alf_idx == (n - 1):
        value[val_idx] = alf[0]
        if val_idx == 0:
            value.insert(0, alf[1])
            return
        val_idx -= 1
        alf_idx = alf.index(value[val_idx])

    # Increment digit by one
    value[val_idx] = alf[alf_idx + 1]


def canonical_code(lengths: List[int], q: int, alf: List[str]) -> List[str]:
    """Returns the canonical prefix code over the alphabet `alf` associated
    to a list of integers.
    """
    # Kraft-McMillan inequality
    assert sum(pow(q, -li) for li in lengths) <= 1

    counter = list(Counter(sorted(lengths)).items())
    code = []
    codeword = []

    # Construct code in lexicographical order
    for (length, count) in counter:
        for _ in range(length - len(codeword)):
            codeword.append(alf[0])
        for _ in range(count):
            code.append("".join(str(d) for d in codeword))
            increment_by_one(codeword, alf)

    # Reorder codewords
    index = 0
    for length in lengths:
        for i in range(index, len(code)):
            if len(code[i]) == length:
                temp = code.pop(i)
                code.insert(index, temp)
                break
        index += 1

    return code

In [4]:
def shannon_code(src: List[Tuple[str, int]]) -> List[str]:
    w = sum(wi for (_, wi) in src)
    lengths = [math.ceil(-math.log2(wi / w)) for (_, wi) in src]

    return canonical_code(lengths, 2, ['0','1'])

# Extra

Funcions extra per a facilitar la realitzacio dels testos d'Atenea.

In [5]:
def maximal_code_missing_lengths(lengths: List[int]) -> List[int]:
    max_len = max(lengths)

    total = pow(2, max_len)
    consumed = sum(pow(2, max_len - li) for li in lengths)
    remaining = total - consumed
    bin_rep = "{:b}".format(remaining)

    result = []
    for i, digit in enumerate(bin_rep):
        if digit == '1':
            result.append(i + 1)

    return result

# Proves

In [6]:
canonical_code([16, 8, 10, 7, 16, 3, 12, 5, 14, 15, 13, 17, 8, 14, 13, 14, 17, 11], 2, ['0', '1'])

['0010110010001110',
 '00101010',
 '0010110000',
 '0010100',
 '0010110010001111',
 '000',
 '001011000110',
 '00100',
 '00101100100000',
 '001011001000110',
 '0010110001110',
 '00101100100100000',
 '00101011',
 '00101100100001',
 '0010110001111',
 '00101100100010',
 '00101100100100001',
 '00101100010']

In [7]:
maximal_code_missing_lengths([9, 8, 6, 9, 9, 6, 8, 7, 10, 10, 5])

[1, 2, 3, 5, 7]

In [8]:
canonical_code([15, 15, 13, 7, 15, 12, 12, 3, 13, 8, 17, 5, 17, 15, 7, 13, 16, 9], 8, ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h'])

['aababacbbaacdaa',
 'aababacbbaacdab',
 'aababacbbaaca',
 'aababaa',
 'aababacbbaacdac',
 'aababacbbaaa',
 'aababacbbaab',
 'aaa',
 'aababacbbaacb',
 'aababaca',
 'aababacbbaacdaeba',
 'aabaa',
 'aababacbbaacdaebb',
 'aababacbbaacdad',
 'aababab',
 'aababacbbaacc',
 'aababacbbaacdaea',
 'aababacba']

In [9]:
text = "él también el nombre, y le cobrase famoso y de estruendo, como convenía a la nueva orden y al nuevo ejercicio que ya profesaba. Y así, después de muchos nombres que formó, borró y quitó, añadió, deshizo y tornó a hacer en su memoria e imaginación, al fin le vino a llamar Rocinante: nombre, a su parecer, alto, sonoro y significativo de lo que había sido cuando fue rocín, antes de lo que ahora era, que era antes y primero de todos los rocines del mundo. Puesto nombre, y tan a su gusto, a su caballo, quiso ponérsele a sí mismo, y en este pensamiento duró otros ocho días, y al cabo se vino a llamar don Quijote; de donde -como queda dicho- tomaron ocasión los autores desta tan verdadera historia"

src = [(' ', 1877), ('!', 1), ("'", 4), (',', 212), ('-', 5), ('.', 37), (':', 6), (';', 21), ('?', 1), ('A', 10), ('B', 4), ('C', 8), ('D', 6), ('E', 7), ('F', 4), ('G', 6), ('H', 2), ('I', 2), ('L', 6), ('M', 8), ('N', 2), ('O', 1), ('P', 6), ('Q', 9), ('R', 5), ('S', 3), ('T', 6), ('U', 1), ('Y', 6), ('a', 1017), ('b', 175), ('c', 302), ('d', 427), ('e', 1067), ('f', 46), ('g', 80), ('h', 92), ('i', 358), ('j', 34), ('l', 498), ('m', 230), ('n', 550), ('o', 734), ('p', 149), ('q', 132), ('r', 508), ('s', 571), ('t', 267), ('u', 391), ('v', 68), ('x', 2), ('y', 122), ('z', 34), ('¡', 1), ('¿', 1), ('á', 31), ('é', 29), ('í', 95), ('ñ', 18), ('ó', 62), ('ú', 7), ('ü', 1)]

code = shannon_code(src)
corr = list(zip((k for (k, _) in src), code))

encode(text, corr)

'101011001011000001001100010100101100011010111010110010110100000110110000001101010010010110001101110001110001000010100010000110000110001001000100100011011100010011110011000101001100010100101010001111010000010100010000101000110000011011111001100111010000001101101010100100100010000100100010010010101000001001000100011011010100000110110110100100010000001000001100001000001101100000011101010000010000010001110010100011011010001010001000001001100000011011000000111010100001000000011101010110001101110100100010111001000101101000001010000100000011000101000100100001001111011100100101001100011011110010100011001010101010000010101110101000001001111101001010001000001010001101111100111110000101011001011110000101000110001001011000010010010011100100011110000110101001001011000110111000110111100010100001000000110001010011001000111010010110101001100010000100011010001110011101010100100010100010001010000100000101110011010101001100010000001010101101000010010100101110101001100010000010100011011111001110010111010

In [10]:
src = [(' ', 1877), ('!', 1), ("'", 4), (',', 212), ('-', 5), ('.', 37), (':', 6), (';', 21), ('?', 1), ('A', 10), ('B', 4), ('C', 8), ('D', 6), ('E', 7), ('F', 4), ('G', 6), ('H', 2), ('I', 2), ('L', 6), ('M', 8), ('N', 2), ('O', 1), ('P', 6), ('Q', 9), ('R', 5), ('S', 3), ('T', 6), ('U', 1), ('Y', 6), ('a', 1017), ('b', 175), ('c', 302), ('d', 427), ('e', 1067), ('f', 46), ('g', 80), ('h', 92), ('i', 358), ('j', 34), ('l', 498), ('m', 230), ('n', 550), ('o', 734), ('p', 149), ('q', 132), ('r', 508), ('s', 571), ('t', 267), ('u', 391), ('v', 68), ('x', 2), ('y', 122), ('z', 34), ('¡', 1), ('¿', 1), ('á', 31), ('é', 29), ('í', 95), ('ñ', 18), ('ó', 62), ('ú', 7), ('ü', 1)]

code = shannon_code(src)
corr = list(zip((k for (k, _) in src), code))

text = "010100011011000110000101000100000110000100000101001011100111110000100110101010010001010001000100110100001010100001000001001111010001110000100100001101100001001010001000010100110010110110110100100111101011100101001000001010001100000110110110010000101010101100011101010100000101011110001100000011000011001000000111010011101000000010000101010000011011100000111110000000011100100100100101001001101100010000101000110001000000101000001101101000010000001100010011000110110110100100010000100101101011000011110001001001000000100111010011001000111100010100001000000110001000001101000011100011001001100000101000100010010110101100001111000100110001010010010011100010011110001010000100000011000001101100000100100001010001100100110001100010000001010001100010101101111010001101001101100001010001000010100001000000110001001100010011011001101000010010100010011110011011000110001011011110000011100110000010001111011110010000101001101000001011100110100010000011000011000100111100100111000111001000101110101001000101000010000001100001101010110000011011000001010111100001000010010010101100110100110001001100010000001010001100010101101011011000011101010110001001101010100111001000000110101011000101011110000001010001101011001110010000100000011011000000101000110110000010101101100010110101000010010001000110100010101100101100000011110011000010111010011110000001001100001010001100100110110101010000010101101100100000010100110011100100000010101010010001001111000011110011000011000011000100111100100111100100111001000110100000110110100001011100101001010100111010110110100100111000010100001000010101100100001101010010010110001101110001100001100001100010011110100011010101001110101001000101010101010001001111010001110101000010000001110001000001111001110100111101011101100110100001111001100001010001110010010100100010000101011001011000000010000011111010010000100101001101111100101010010001000001101010000000110111000100000111000101010101111010100101101000101000010000001100010010000101000110010011000110001000000101000110001001000010100011001001100011000011011100100000100110001001101000101001100010100101010001111010010001000010100010001001100010011010001000111000000110110101000001010110010110000010011110100011100000111110100101000100000011011111001101000010101000010110011011110011000011110101101101000011010100100101100011011100011000100100010001101010010010001011010100100101010101000101000100000100110101111101001010001000010011110111001001001001000001110001010001100100000010100100010010010101000101010101100001110011110011011000011000010100011000100101001001101001101110001000010100001000000110000101000111001000110000100111000100111100110001010000100000101110101100101101000100111000101000111010010001000001111010110101001001000100000010011011001100011011110001010000100000011000101001101000000110111100110000101000110001001000010100011001001100011000011011100100000001001101010100010011011001100011100010000101000100001100010000010100001000000110000011011100010000001101101100110010001101100100001101111101010101000100111110000001101111000001101111100110001010001100100001001011000010100010001001111100000011011111001100100000001101101000011100010101010111101010010110100010100001000000111000100001001011000001010001001101010100100000011111000000001111001110101101000100011100000011011111001100010010100100100010000100101100000101000100111100110001010110010110000010011000101001011000110101110101100101101000001101100000011010100100101100011011100011100010000101000100001100001100010010001001000110111000100111100110001010011000101001010100011110100000101000100001010001100000110111110011001110100000011011010101001001000100001001000100100101010000010010001000110110101000001101101101001000100000010000011000010000011011000000111010100000100000100011100101000110110100010100010000010011000000110110000001110101000010000000111010101100011011101001000101110010001011010000010100001000000110001010001001000010011110111001001010011000110111100101000110010101010100000101011101010000010011111010010100010000010100011011111001111100001010110010111100001010001100010010110000100100100111001000111100001101010010010110001101110001101111000101000010000001100010100110010001110100101101010011000100001000110100011100111010101001000101000100010100001000001011100110101010011000100000010101011010000100101001011101010011000100000101000110111110011100101110101011101000001010001000100110010001110011011010100100000100001001110001010010000110111000000110110100001111100000001001010011100101010001110010110010000001100001011100101001010100111010110110100101001000101110101001011011000100000010011000001010011001011011010000110000110001010100001011011010100000001000001100011000010100101001001110000101011110010010010010001011011010010011011001100011101011010100000110101001001011000110111000111000100000010000011111000000010011110010011100011100100001101110100010"

decode(text, corr)

'della, la diputó y tuvo por celada finísima de encaje. Fue luego a ver su rocín, y, aunque tenía más cuartos que un real y más tachas que el caballo de Gonela, que tantum pellis et ossa fuit, le pareció que ni el Bucéfalo de Alejandro ni Babieca el del Cid con él se igualaban. Cuatro días se le pasaron en imaginar qué nombre le pondría; porque, según se decía él a sí mesmo, no era razón que caballo de caballero tan famoso, y tan bueno él por sí, estuviese sin nombre conocido; y ansí, procuraba acomodársele de manera que declarase quién había sido, antes que fuese de caballero andante, y lo que era entonces; pues estaba muy puesto en razón que, mudando su señor estado, mudase él también el nombre, y le cobrase famoso y de estruendo, como convenía a la nueva orden y al nuevo ejercicio que ya profesaba. Y así, después de muchos nombres que formó, borró y quitó, añadió, deshizo y tornó a hacer en su memoria e imaginación, al fin le vino a llamar Rocinante: nombre, a su parecer,'